In [1]:
!pip install -q torch torchvision numpy

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
from collections import OrderedDict
import itertools
import sys
import time

sys.path.append("..")

from prunenn.data import *
from prunenn.models import *
from prunenn.trainer import *
from prunenn.pruner import *
from prunenn.run_pruner import *

In [2]:
dataset = 'mnist'

# Select model and data loaders based on data set
if dataset == 'mnist':
    model = MNISTNet()
    loaders = get_mnist_loaders()
elif dataset == 'cifar':
    model = CIFARNet()
    loaders = get_cifar_loaders() 
train_loader, test_loader = loaders

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [3]:
'''
Pretraining full model
'''
epoch_range = 1
            
for epoch in range(epoch_range):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.532043
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.391521
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.169306
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.192877
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.081172
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.108842
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.033457
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.021315
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.084689
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.045041

Test set: Average loss: 0.0496, Accuracy: 9859/10000 (99%)



In [4]:
pruner_model = Pruner(model, function="corrs")
pruner_model.to_train = False
t0 = time.time()
acc = test(pruner_model, test_loader)
t1 = time.time()
print("testing time", (t1-t0))
sum(p.numel() for p in model.parameters() if p.requires_grad)


Test set: Average loss: 0.0496, Accuracy: 9859/10000 (99%)

testing time 3.1304309368133545


25670

In [ ]:
thresholds = np.flip(np.logspace(np.log10(0.7), np.log10(0.95), num=10), axis=0)
sacrifice = 0.01
pruner_model = prune_loop(pruner_model, thresholds, sacrifice, loaders, optimizer)


Test set: Average loss: 0.0496, Accuracy: 9859/10000 (99%)

--- threshold  0.95  ---
Train Epoch: 0 [0/60000 (0%)]	Loss: 0.048976


/Applications/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Applications/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/rohitbandaru/Desktop/PruneNN/prunenn/pruner.py:132: RuntimeWarning: invalid value encountered in less_equal
  nodes_to_prune = np.where((self.thres <= corrs[:, :]))


Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.020210
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.011674
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.080999
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.010653
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.065219
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.051694
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.017175
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.042880
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.016537

Test set: Average loss: 0.0372, Accuracy: 9884/10000 (99%)

Removing 0 nodes from conv2d layer
Removing 0 nodes from batchnorm layer
Removing 2 nodes from conv2d layer
Removing 2 nodes from batchnorm layer
Removing 0 nodes from conv2d layer
Removing 0 nodes from batchnorm layer

Test set: Average loss: 0.0644, Accuracy: 9810/10000 (98%)

--- threshold  0.9183060200093368  ---
Train Epoch: 0 [0/60000 (0%)]	Loss: 0.009580
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.089386
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.115203
Train Epoch: 0 [19200/60000

In [ ]:
t0 = time.time()
acc = test(model, test_loader)
t1 = time.time()
print("testing time", (t1-t0))
sum(p.numel() for p in pruner_model.parameters() if p.requires_grad)